## Four Corners program
Name: Divya Natekar

Net ID: dyn2009

NYU ID: N19974330

In [1]:
import open3d as o3d
import numpy as np
from collections import defaultdict

# Load the original point cloud
file_path = "jay-willoughby.ply"
pcd = o3d.io.read_point_cloud(file_path)
points = np.asarray(pcd.points)

if len(points) == 0:
    raise ValueError("Loaded point cloud is empty!")

# Define scanner corners
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()
z_lift = 1.5

scanner_positions = [
    np.array([min_bound[0] + 1.0, min_bound[1] + 1.0, min_bound[2] + z_lift]),
    np.array([max_bound[0] - 1.0, min_bound[1] + 1.0, min_bound[2] + z_lift]),
    np.array([max_bound[0] - 1.0, max_bound[1] - 1.0, min_bound[2] + z_lift]),
    np.array([min_bound[0] + 1.0, max_bound[1] - 1.0, min_bound[2] + z_lift])
]

for i, scanner_pos in enumerate(scanner_positions):
    vectors = points - scanner_pos
    norms = np.linalg.norm(vectors, axis=1)
    unit_vectors = vectors / norms[:, None]

    rounded_vectors = np.round(unit_vectors * 1000).astype(int)
    direction_bins = defaultdict(list)
    for idx, dir_vec in enumerate(rounded_vectors):
        direction_bins[tuple(dir_vec)].append((idx, norms[idx]))

    retain_indices = {max(bin, key=lambda x: x[1])[0] for bin in direction_bins.values()}
    all_indices = set(range(len(points)))
    removed_indices = list(all_indices - retain_indices)
    retain_indices = list(retain_indices)

    retained_pcd = pcd.select_by_index(retain_indices)
    removed_pcd = pcd.select_by_index(removed_indices)

    # Grayscale retained
    grey = np.mean(np.asarray(retained_pcd.colors), axis=1)
    retained_pcd.colors = o3d.utility.Vector3dVector(np.tile(grey[:, None], 3))
    
    # Green for removed
    removed_pcd.paint_uniform_color([0, 1, 0])

    o3d.io.write_point_cloud(f"corner{i+1}_retained_final.ply", retained_pcd)
    o3d.io.write_point_cloud(f"corner{i+1}_removed_final.ply", removed_pcd)
